# Import libraries

In [5]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
import datetime as dt
import plotly.graph_objects as go_offline
from pandas_datareader import data

You should consider upgrading via the 'c:\users\razer\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\razer\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


# Import data

In [6]:
df_portfolio = pd.read_csv('portfolio.csv')
df_portfolio

,stock,qty,price,current_prices,P&L
0,AAPL,15.0,126.67,146.92,304.0
1,TSLA,18.0,711.11,774.39,1139.0
2,SPY,10.0,300.00,443.91,1439.0


In [7]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [8]:
df_input = pd.read_csv('input.csv')
df_input

,date,stock,action,qty,price
0,26-10-20,Cash,Deposit,1,1000.0
1,27-10-20,Cash,Deposit,1,1000.0
2,28-10-20,AAPL,BUY,10,100.0
3,29-10-20,TSLA,BUY,10,600.0
4,30-10-20,SPY,BUY,10,300.0
5,31-10-20,Cash,Rebate,1,2.0
6,01-11-20,Cash,Rebate,1,2.0
7,02-11-20,Cash,Rebate,1,2.0
8,03-11-20,AAPL,SELL,-5,120.0
9,04-11-20,TSLA,SELL,-2,700.0


# Annualised Returns for each stock

In [9]:
tickers = list(df_portfolio['stock'])
print(tickers)
today = dt.date.today()
year_ago = dt.date.today() - dt.timedelta(days=365)

start_date = year_ago.strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")


panel_data = data.DataReader(tickers, 'yahoo', start_date, end_date)

close_price = panel_data['Close']
adj_close = panel_data['Adj Close']

['AAPL', 'TSLA', 'SPY']


In [10]:
ahv = np.sqrt(np.log(close_price/close_price.shift(1)).var()) * np.sqrt(252)
ahv_pct = ahv*100

In [11]:
ahv_pct = pd.DataFrame(ahv_pct)
ahv_pct.rename(columns = {0:'Annualized Historical Volatility'}, inplace=True)
ahv_pct = ahv_pct.rename_axis('Ticker').reset_index()

In [17]:
fig = px.bar(ahv_pct,
       x='Ticker',
       y='Annualized Historical Volatility',
       title="Annualized Historical Volatility of individual stocks in portfolio",
      color="Ticker")

fig

In [ ]:
df_input

In [13]:
dates = df_input['date']
pnl = [0]
portfolio_dict = {}

## The logic is flawed here, it is only assuming 1 action per day (fix later)
## The code doesnt handle repeating the same stock (fix later)
for i in range(len(df_input)):
    if df_input['action'][i] == 'BUY':
        portfolio_dict[df_input['stock'][i]] = [df_input['qty'][i], df_input['price'][i]]
    if df_input['action'][i] != 'SELL':
        pnl += [pnl[-1]+0]
    elif df_input['action'][i] == 'SELL':
        pnl += [pnl[-1]+abs(df_input['qty'][i])*(df_input['price'][i]-portfolio_dict[df_input['stock'][i]][1])]

In [ ]:
pnl = pnl[1:]

In [14]:
daily_pnl = pd.DataFrame(dates)
daily_pnl['Daily PnL'] = pd.DataFrame(pnl)
daily_pnl['date'] = pd.to_datetime(daily_pnl['date'], format='%d-%m-%y')
daily_pnl

,date,Daily PnL
0,2020-10-26,0.0
1,2020-10-27,0.0
2,2020-10-28,0.0
3,2020-10-29,0.0
4,2020-10-30,0.0
5,2020-10-31,0.0
6,2020-11-01,0.0
7,2020-11-02,0.0
8,2020-11-03,0.0
9,2020-11-04,100.0


In [15]:
px.line(daily_pnl,
        x = 'date',
        y = 'Daily PnL',
       title="Daily PnL")

In [16]:
daily_pnl

,date,Daily PnL
0,2020-10-26,0.0
1,2020-10-27,0.0
2,2020-10-28,0.0
3,2020-10-29,0.0
4,2020-10-30,0.0
5,2020-10-31,0.0
6,2020-11-01,0.0
7,2020-11-02,0.0
8,2020-11-03,0.0
9,2020-11-04,100.0
